In [2]:
import bs4
import pandas as pd
import numpy as np
from src.util.util_dicts import nba_teams_post_2000, home_map
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import random

## Scraping Player logs

#### Nov 14th 

Thought that i had player logs fully downloaded, turns out was hitting the bbref bot catcher and timed out for large stretches. Next steps
- Clean up player logs folder (done)
- Fix player write to csv logic (done)
- add check for bot (done)
- Notebooks for specialized things:
- - Open and pull player log for given season, first, last name (done)
  - Open and pause with player log open (done)
 

#### Nov 8th Thinking

- Struggling to complete player bbref codes list, manual complete the last X names?


#### Nov 6th Thinking

Another long few days without coding, need to regroup and have something for wednesday
- Need Player list in GCP
- Need player bbref code in GCP
- Need loop for players to be pulled to CSV
- Need to clean up git
- Need to clean up readme
- Logging standard file

#### Nov 1st Thinking

Been a few days since progress, need to commit to git at some point, make sure i am on correct branches

- Advanced stats cleaned and added to pycharm
- For each nba player loop
- Start saving to CSV by player
- Thinking on how to handle players who's careers started prior to 2000 (Potentially go back and pull all games prior to 2000 as well, different formatting)


#### Oct 26th Thinking

Did not get as much done yesterday, now have NBA players list as CSV, full and post 2000. Goal for today:
- Pull and clean advanced stats
- Append to player game log df (s)
- Git cleaning (s)
- Create BQ table and write NBA player list (s)
- Create BQ table and write post 2000 nba (s)

#### Oct 25th Thinking

Lots of progress yesterday, need to start thinking about saving data to cloud and cleaning up repository
For today, consider doing:
- Clean up repository
- Investigate advanced player stats
- Write all nba/aba players to table (consider keeping additional context)
- Investigate edge cases in names (Marvin Bagley III, Dennis Smith Jr). Know names with more than one ' ' will be an issue
- Start thinking about looping over player names, perhaps one letter at a time, have a csv for each letter that can easily be opened and added to BQ

#### Oct 24th Thinking

- Got way to search player game logs by season, next need to clean up the returned table into a single clean df
- Showing need to extract all selenium related functions into their own specific python file in the future (selenium_helper.py)


#### Oct 23rd Thinking

Need to start with some base functions that will get called a ton, these will include:

- Open, navigate, pull single year player game log (input player, year, output raw HTML table)
- Take HTML table, parse into rough df that needs formatting
- Take rough df and clean, output single year player stats

Once this is ready, create list of players that played since 2000, could be its own table? (player name, first year, last year)

For now lets use LBJ as our first player to put this in practice on

In [5]:
first = "Lebron"
last = "James"

In [7]:
# TODO extract into player specific helpers/util

def player_name_to_bbref_code(first, last):
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()

In [8]:
# TODO extract into player specific helpers/util


def get_player_season_games_log(first, last, season):
    player_code = player_name_to_bbref_code(first, last)
    url = f"https://www.basketball-reference.com/players/{player_code}/gamelog/{season}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_basic')
    df = pd.read_html(str(table))[0]
    return df

In [9]:
# TODO extract into selenium specific helper/util

# Options for selenium
options = Options()
options.page_load_strategy = 'eager'  # Faster load so it does not wait for video ads to render
options.add_argument("--headless")  # Run Chrome in headless mode

def get_webpage_html(url):
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    html = driver.page_source
    driver.close()
    return html

In [266]:
df = get_player_season_games_log(first, last, 2022)

In [267]:
df = clean_player_games_log(df)

KeyError: 1

In [40]:
df_original = df.copy()

In [16]:
# Specific for the player game log table

# TODO, investigate try/catch for this in case missing values?
def column_type_conversion(df):
    df.loc[:, 'tm_game_num'] = df['tm_game_num'].astype(int)
    df.loc[:, 'p_game_num'] = df['p_game_num'].astype(int)
    df.loc[:, 'home'] = df['home'].astype(bool)
    df.loc[:, 'GS'] = df['GS'].astype(bool)
    df.loc[:, 'FG'] = df['FG'].astype(int)
    df.loc[:, 'FGA'] = df['FGA'].astype(int)
    df.loc[:, 'FG%'] = df['FG%'].astype(float)
    df.loc[:, '3P'] = df['3P'].astype(int)
    df.loc[:, '3PA'] = df['3PA'].astype(int)
    df.loc[:, '3P%'] = df['3P%'].astype(float)
    df.loc[:, 'FT'] = df['FT'].astype(int)
    df.loc[:, 'FTA'] = df['FTA'].astype(int)
    df.loc[:, 'FT%'] = df['FT%'].astype(float)
    df.loc[:, 'ORB'] = df['ORB'].astype(int)
    df.loc[:, 'DRB'] = df['DRB'].astype(int)
    df.loc[:, 'TRB'] = df['TRB'].astype(int)
    df.loc[:, 'AST'] = df['AST'].astype(int)
    df.loc[:, 'STL'] = df['STL'].astype(int)
    df.loc[:, 'BLK'] = df['BLK'].astype(int)
    df.loc[:, 'TOV'] = df['TOV'].astype(int)
    df.loc[:, 'PF'] = df['PF'].astype(int)
    df.loc[:, 'PTS'] = df['PTS'].astype(int)
    df.loc[:, 'GmSc'] = df['GmSc'].astype(float)
    df.loc[:, 'PM'] = df['PM'].astype(int)
    df.loc[:, 'age_yrs'] = df['age_yrs'].astype(int)
    df.loc[:, 'age_days'] = df['age_days'].astype(int)
    df.loc[:, 'score_diff'] = df['score_diff'].astype(int)    
    # if type(df.date[1]) == str:
    df.loc[:, 'date'] = pd.to_datetime(df['date'], format='mixed')
    return df

In [17]:
col_order = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 'Tm', 
             'Opp', 'home','result', 'score_diff', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', 
             '3PA', '3P%', 'FT','FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 
             'TOV', 'PF','PTS', 'GmSc', 'PM',] # Specific column order because cleaner

home_map = {np.nan: 1, "@": 0}

def extract_date(dt): # To create gameID column
    return dt.date().strftime("%Y%m%d")

def clean_player_games_log(df):
    
    df = df[df['G'] != 'G'].copy() # remove formatting rows
    column_rename = {'Rk':'tm_game_num', 'G':'p_game_num', 'Date': 'date', 'Age':'age','Unnamed: 5':'home', 'Unnamed: 7':'result','+/-':'PM'} 
    df.rename(columns=column_rename, inplace=True) # Rename necessary columns

    df = df[df['GS'] != 'Inactive'].copy() # Remove rows where the player was inactive for the game
    df = df[df['GS'] != 'Did Not Dress'].copy() # Remove rows where player was not active (for player stats research would want to keep, but for ML not needed
    df = df[df['GS'] != 'Did Not Play'].copy() # Remove rows where player was not active, mutual decision? lol
    df = df[df['GS'] != 'Not With Team'].copy() # Remove rows where player was not with team?


    # NOTE: Will be more of these to drop, e.g. did not dress
    
    df.loc[:, 'home'] = df['home'].replace(home_map) # Map home to a bool column, true = home

    # Split compound columns
    df[['age_yrs', 'age_days']] = df['age'].str.split('-', expand=True)
    df[['result', 'score_diff']] = df['result'].str.split('(', expand=True)
    df['score_diff'] = df['score_diff'].str.rstrip(')')

    df = df.drop(columns = ['age'])

    
    df = column_type_conversion(df)

    df['gameID'] = df['date'].apply(extract_date) + df["Tm"] + df["Opp"]

    df = df[col_order]
    return df
    

In [124]:
df = df_original.copy()

In [125]:
df = clean_player_games_log(df)

In [269]:
# Test to import all lebron james seasons to single df

df = pd.DataFrame()
first = "Pascal"
last = "Siakam"

for year in range(2017, 2024):
    temp = get_player_season_games_log(first, last, year)
    temp = clean_player_games_log(temp)
    df = pd.concat([df, temp])  
    print('appended ', year)

df

appended  2017
appended  2018
appended  2019
appended  2020
appended  2021
appended  2022
appended  2023


,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,Opp,home,result,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,PM
0,20161026TORDET,1,1,2016-10-26 00:00:00,22,207,TOR,DET,True,W,...,7,9,0,1,0,0,2,4,7.1,+16
1,20161028TORCLE,2,2,2016-10-28 00:00:00,22,209,TOR,CLE,True,L,...,0,3,0,1,0,2,3,4,1.9,-7
2,20161031TORDEN,3,3,2016-10-31 00:00:00,22,212,TOR,DEN,True,W,...,3,4,0,0,0,0,3,2,0.0,-4
3,20161102TORWAS,4,4,2016-11-02 00:00:00,22,214,TOR,WAS,False,W,...,3,3,0,1,0,1,3,4,1.7,-6
4,20161104TORMIA,5,5,2016-11-04 00:00:00,22,216,TOR,MIA,True,W,...,2,3,0,0,0,1,0,8,5.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20230331TORPHI,77,67,2023-03-31 00:00:00,28,363,TOR,PHI,False,L,...,4,4,7,0,0,2,2,14,8.4,NaN
80,20230402TORCHO,78,68,2023-04-02 00:00:00,29,0,TOR,CHO,False,W,...,5,7,7,2,0,0,1,36,35.0,+16
81,20230404TORCHO,79,69,2023-04-04 00:00:00,29,2,TOR,CHO,False,W,...,11,14,4,2,2,2,4,22,21.6,+5
82,20230405TORBOS,80,70,2023-04-05 00:00:00,29,3,TOR,BOS,False,L,...,9,11,4,2,0,1,3,28,26.8,+1


In [146]:
df_lebron = df.copy()

In [147]:
# Test to import all Jokic seasons to single df

df = pd.DataFrame()
first = "Nikola"
last = "Jokic"

for year in range(2016, 2024):
    temp = get_player_season_games_log(first, last, year)
    temp = clean_player_games_log(temp)
    df = pd.concat([df, temp])  
    print('appended ', year)

df_jokic = df.copy()
df_jokic

appended  2016
appended  2017
appended  2018
appended  2019
appended  2020
appended  2021
appended  2022
appended  2023


,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,Opp,home,result,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,PM
0,20151028DENHOU,1,1,2015-10-28 00:00:00,20,251,DEN,HOU,False,W,...,0,0,0,0,0,1,0,2,0.7,+1
1,20151030DENMIN,2,2,2015-10-30 00:00:00,20,253,DEN,MIN,True,L,...,6,9,1,0,0,2,3,10,7.8,+10
2,20151101DENOKC,3,3,2015-11-01 00:00:00,20,255,DEN,OKC,False,L,...,1,4,0,0,0,1,2,8,3.9,-2
4,20151105DENUTA,5,4,2015-11-05 00:00:00,20,259,DEN,UTA,True,L,...,2,3,0,0,1,1,1,5,2.8,-6
5,20151106DENGSW,6,5,2015-11-06 00:00:00,20,260,DEN,GSW,False,L,...,2,4,1,1,1,1,1,4,4.2,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,20230322DENWAS,73,65,2023-03-22 00:00:00,28,31,DEN,WAS,False,W,...,7,12,7,3,0,2,1,31,35.7,+28
76,20230325DENMIL,74,66,2023-03-25 00:00:00,28,34,DEN,MIL,True,W,...,5,6,11,1,0,3,2,31,27.7,+19
77,20230327DENPHI,75,67,2023-03-27 00:00:00,28,36,DEN,PHI,True,W,...,14,17,12,0,2,3,3,25,31.6,+11
81,20230404DENHOU,79,68,2023-04-04 00:00:00,28,44,DEN,HOU,False,L,...,8,10,4,2,3,8,1,14,9.6,-21


In [270]:
df_pascal = df.copy()

In [284]:
pascal_recent = df_pascal[(df_pascal['date'] >= pd.to_datetime('2021-10-01')) & (df_pascal['home'])]

In [287]:
pascal_recent = df_pascal[(df_pascal['date'] >= pd.to_datetime('2022-10-01')) & (df_pascal['home'])]
pascal_bt = len(pascal_recent[pascal_recent['PTS'] >= 20])
pascal_total = pascal_recent.shape[0]
print(f"Pascal games over threshold {pascal_bt} out of {pascal_total}, above threshold {pascal_bt/pascal_total}% of the time")


Pascal games over threshold 27 out of 35, above threshold 0.7714285714285715% of the time


In [ ]:
jokic_bt = len(jokic_recent[jokic_recent['PTS'] > 20])
jokic_total = jokic_recent.shape[0]
lebron_bt = len(df_lebron[df_lebron['PTS'] < 15])
lebron_total = df_lebron.shape[0]
print(f"Jokic games under threshold {jokic_bt} out of {jokic_total}, below threshold {jokic_bt/jokic_total}% of the time")
print(f"Lebron games under threshold {lebron_bt} out of {lebron_total}, below threshold {lebron_bt/lebron_total}% of the time")

In [154]:
jokic_recent = df_jokic[df_jokic['date'] >= pd.to_datetime('2021-01-01')]

In [155]:
jokic_bt = len(df_jokic[df_jokic['PTS'] < 15])
jokic_total = df_jokic.shape[0]
lebron_bt = len(df_lebron[df_lebron['PTS'] < 15])
lebron_total = df_lebron.shape[0]
print(f"Jokic games under threshold {jokic_bt} out of {jokic_total}, below threshold {jokic_bt/jokic_total}% of the time")
print(f"Lebron games under threshold {lebron_bt} out of {lebron_total}, below threshold {lebron_bt/lebron_total}% of the time")

Jokic games under threshold 173 out of 596, below threshold 0.2902684563758389% of the time
Lebron games under threshold 59 out of 1421, below threshold 0.04152005629838142% of the time


In [156]:
jokic_bt = len(jokic_recent[jokic_recent['PTS'] < 15])
jokic_total = jokic_recent.shape[0]
lebron_bt = len(df_lebron[df_lebron['PTS'] < 15])
lebron_total = df_lebron.shape[0]
print(f"Jokic games under threshold {jokic_bt} out of {jokic_total}, below threshold {jokic_bt/jokic_total}% of the time")
print(f"Lebron games under threshold {lebron_bt} out of {lebron_total}, below threshold {lebron_bt/lebron_total}% of the time")

Jokic games under threshold 18 out of 211, below threshold 0.08530805687203792% of the time
Lebron games under threshold 59 out of 1421, below threshold 0.04152005629838142% of the time


## Scrape NBA Players list

Need to create a list of all NBA players that have played in the league since 2000, use the BBref player directory, pull each last name letter (A->Z), clean the table, will have list of all players in nba/aba history along with their first and last year in the league

In [179]:
def get_player_years_active(letter):
    url = f"https://www.basketball-reference.com/players/{letter}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'players')
    df = pd.read_html(str(table))[0]
    return df

In [224]:
def clean_players_table(df):
    df = df[df['From'] != 'From']
    cols_to_drop = ['Colleges', 'Birth Date', 'Wt', 'Ht', 'Pos']
    df = df.drop(columns = cols_to_drop)
    df[['first_name', 'last_name']] = df['Player'].str.split(' ', n=1, expand=True)
    df['last_name'] = df['last_name'].str.rstrip('*')
    df['From'] = df['From'].astype(int)
    df['To'] = df['To'].astype(int)
    df = df[['first_name', 'last_name','From','To']]
    return df

In [260]:
def clean_players_table_all(df):
    df = df[df['From'] != 'From'].copy()
    cols_to_rename = {'Birth Date': 'birth_date', 'Colleges': 'college'}
    df.rename(columns = cols_to_rename, inplace=True)
    df[['first_name', 'last_name']] = df['Player'].str.split(' ', n=1, expand=True)
    df['hall_of_fame'] = df['last_name'].str.endswith('*') # Keep track of hall of fame as a bool column
    df['last_name'] = df['last_name'].str.rstrip('*')
    df['From'] = df['From'].astype(int)
    df['To'] = df['To'].astype(int)
    df['Wt'] = df['Wt'].astype(int)
    if type(df.birth_date[1]) == str:
        df.loc[:, 'birth_date'] = pd.to_datetime(df['birth_date'], format='mixed')
    df = df[['first_name', 'last_name', 'From', 'To', 'Pos', 'Ht', 'Wt', 'birth_date', 'college', 'hall_of_fame']]
    return df


In [250]:
test = get_player_years_active('b')

In [251]:
test

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges
0,Chris Babb,2014,2014,G,6-5,225,"February 14, 1990","Penn State, Iowa State"
1,Luke Babbitt,2011,2018,F,6-9,225,"June 20, 1989",Nevada
2,Miloš Babić,1991,1992,C-F,7-0,240,"November 23, 1968",Tennessee Technological University
3,Johnny Bach,1949,1949,F-G,6-2,180,"July 10, 1924",Fordham
4,Dwayne Bacon,2018,2021,G,6-6,221,"August 30, 1995",Florida State
...,...,...,...,...,...,...,...,...
509,Will Bynum,2006,2015,G,6-0,185,"January 4, 1983","Arizona, Georgia Tech"
510,Walt Byrd,1970,1970,F,6-7,205,NaN,Temple
511,Marty Byrnes,1979,1983,F,6-7,215,"April 30, 1956",Syracuse
512,Tommy Byrnes,1947,1951,F-G,6-3,175,"February 19, 1923",Seton Hall


In [261]:
df = test.copy()
df = clean_players_table_all(df)

In [262]:
df[df['hall_of_fame'] == True]

,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame
49,Charles,Barkley,1985,2000,F,6-6,252,1963-02-20 00:00:00,Auburn,True
51,Don,Barksdale,1952,1955,F-C,6-6,200,1923-03-31 00:00:00,UCLA,True
78,Rick,Barry,1966,1980,F,6-7,205,1944-03-28 00:00:00,Miami (FL),True
107,Elgin,Baylor,1959,1972,F,6-5,225,1934-09-16 00:00:00,"College of Idaho, Seattle University",True
123,Zelmo,Beaty,1963,1975,C,6-9,225,1939-10-25 00:00:00,Prairie View A&M University,True
144,Walt,Bellamy,1962,1975,C,6-11,225,1939-07-24 00:00:00,Indiana,True
191,Dave,Bing,1967,1978,G,6-3,180,1943-11-24 00:00:00,Syracuse,True
196,Larry,Bird,1980,1992,F,6-9,220,1956-12-07 00:00:00,Indiana State University,True
278,Chris,Bosh,2004,2016,F-C,6-11,235,1984-03-24 00:00:00,Georgia Tech,True
316,Bill,Bradley,1968,1977,F-G,6-5,205,1943-07-28 00:00:00,Princeton,True


In [185]:
players_df = pd.DataFrame()

for i in range(ord('a'), ord('z')+1):
    if(chr(i) == 'x'):
        continue
    temp = get_player_years_active(chr(i))
    players_df = pd.concat([players_df, temp])
    num = random.randint(1, 3) # Otherwise might have basketball ref find out you are a bot
    time.sleep(num)

players_df = clean_players_table(players_df)
players_df = players_df[(players_df['From'] > 2000) | (players_df['To'] > 2000)]
players_df

a
sleeping 4
b
sleeping 5
c
sleeping 1
d
sleeping 2
e
sleeping 1
f
sleeping 2
g
sleeping 4
h
sleeping 1
i
sleeping 3
j
sleeping 2
k
sleeping 2
l
sleeping 2
m
sleeping 5
n
sleeping 3
o
sleeping 5
p
sleeping 3
q
sleeping 5
r
sleeping 3
s
sleeping 5
t
sleeping 5
u
sleeping 2
v
sleeping 1
w
sleeping 4
x
y
sleeping 2
z
sleeping 3


## Start Scraping NBA Player Logs

#### Have completed so far
- Scrape game log for given player + season combo
- Script to pull all NBA players, create df -> csv of all players that played in 2000s

#### Need To Complete
- Investigate advanced stats appending
- For loop logic, open csv, for each player create list of years to iterate over, scrape each year for given player, append to DF

## Scrape Advanced Stats Games Log

- Pull for single player (done)
- Clean DF (done)
- Function to pull both
- Function to concat df together

In [1]:
# TODO extract into player specific helpers/util

def player_name_to_bbref_code(first, last):
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()

In [3]:
# TODO extract into player specific helpers/util


def get_player_adv_season_games_log(first, last, season):
    player_code = player_name_to_bbref_code(first, last)
    url = f"https://www.basketball-reference.com/players/{player_code}/gamelog-advanced/{season}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_advanced')
    df = pd.read_html(str(table))[0]
    return df

In [5]:
# Specific for the player game log table

# TODO, investigate try/catch for this in case missing values?
def adv_column_type_conversion(df):
    df.loc[:, 'tm_game_num'] = df['tm_game_num'].astype(int)
    df.loc[:, 'p_game_num'] = df['p_game_num'].astype(int)
    df.loc[:, 'home'] = df['home'].astype(bool)
    df.loc[:, 'GS'] = df['GS'].astype(bool)
    df.loc[:, 'TS%'] = df['TS%'].astype(float)
    df.loc[:, 'eFG%'] = df['eFG%'].astype(float)
    df.loc[:, 'ORB%'] = df['ORB%'].astype(float)
    df.loc[:, 'DRB%'] = df['DRB%'].astype(float)
    df.loc[:, 'TRB%'] = df['TRB%'].astype(float)
    df.loc[:, 'AST%'] = df['AST%'].astype(float)
    df.loc[:, 'STL%'] = df['STL%'].astype(float)
    df.loc[:, 'BLK%'] = df['BLK%'].astype(float)
    df.loc[:, 'TOV%'] = df['TOV%'].astype(float)
    df.loc[:, 'USG%'] = df['USG%'].astype(float)
    df.loc[:, 'ORtg'] = df['ORtg'].astype(int)
    df.loc[:, 'DRtg'] = df['DRtg'].astype(int)
    df.loc[:, 'GmSc'] = df['GmSc'].astype(float)
    df.loc[:, 'BPM'] = df['BPM'].astype(float)
    df.loc[:, 'age_yrs'] = df['age_yrs'].astype(int)
    df.loc[:, 'age_days'] = df['age_days'].astype(int)
    df.loc[:, 'score_diff'] = df['score_diff'].astype(int)    
    if type(df.date[1]) == str:
        df.loc[:, 'date'] = pd.to_datetime(df['date'], format='mixed')
    return df

In [12]:
first = "Lebron"
last = "James"
season = 2005

df_adv = get_player_adv_season_games_log(first, last, season)
df_adv = clean_player_adv_games_log(df)



,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,home,Opp,result,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,BPM
0,20041103CLEIND,1,1,2004-11-03 00:00:00,19,309,CLE,True,IND,L,...,6.3,33.5,1.1,4.6,19.8,30.3,103,105,20.9,7.4
1,20041104CLEMIA,2,2,2004-11-04 00:00:00,19,310,CLE,False,MIA,L,...,8.5,38.9,1.3,0.0,4.3,26.0,107,109,17.8,9.7
2,20041106CLEMIL,3,3,2004-11-06 00:00:00,19,312,CLE,False,MIL,L,...,10.4,32.9,3.6,0.0,11.9,34.6,104,103,23.8,9.2
3,20041109CLEATL,4,4,2004-11-09 00:00:00,19,315,CLE,False,ATL,W,...,13.9,17.4,4.7,3.2,11.9,25.8,111,83,22.5,8.7
4,20041110CLEPHO,5,5,2004-11-10 00:00:00,19,316,CLE,True,PHO,W,...,10.6,28.0,3.1,4.1,9.3,29.7,130,95,36.1,16.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,20050414CLENYK,78,76,2005-04-14 00:00:00,20,105,CLE,True,NYK,L,...,22.4,28.6,2.6,3.9,11.4,36.3,98,97,19.2,8.7
81,20050415CLEWAS,79,77,2005-04-15 00:00:00,20,106,CLE,False,WAS,L,...,7.5,23.1,1.1,0.0,2.7,32.1,121,131,25.7,5.5
82,20050417CLEDET,80,78,2005-04-17 00:00:00,20,108,CLE,False,DET,L,...,12.5,31.7,0.0,1.5,2.4,42.6,110,111,24.5,14.3
84,20050419CLEBOS,81,79,2005-04-19 00:00:00,20,110,CLE,True,BOS,W,...,10.5,22.9,5.8,0.0,9.3,31.0,121,92,25.7,8.4


In [ ]:
['Rk', 'G', 'Date', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7', 'GS',
       'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'

In [19]:
col_order_adv = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 'Tm', 'home', 'Opp', 
             'result', 'score_diff', 'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 
             'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'] # Specific column order because cleaner

home_map = {np.nan: 1, "@": 0}

def extract_date(dt): # To create gameID column
    return dt.date().strftime("%Y%m%d")

def clean_player_adv_games_log(df):
    
    df = df[df['G'] != 'G'].copy() # remove formatting rows
    column_rename = {'Rk':'tm_game_num', 'G':'p_game_num', 'Date': 'date', 'Age':'age','Unnamed: 5':'home', 'Unnamed: 7':'result'} 
    df.rename(columns=column_rename, inplace=True) # Rename necessary columns

    df = df[df['GS'] != 'Inactive'].copy() # Remove rows where the player was inactive for the game
    df = df[df['GS'] != 'Did Not Dress'].copy() # Remove rows where player was not active (for player stats research would want to keep, but for ML not needed
    df = df[df['GS'] != 'Did Not Play'].copy() # Remove rows where player was not active, mutual decision? lol
    df = df[df['GS'] != 'Not With Team'].copy() # Remove rows where player was not with team?


    # NOTE: Will be more of these to drop, e.g. did not dress
    
    df.loc[:, 'home'] = df['home'].replace(home_map) # Map home to a bool column, true = home

    # Split compound columns
    df[['age_yrs', 'age_days']] = df['age'].str.split('-', expand=True)
    df[['result', 'score_diff']] = df['result'].str.split('(', expand=True)
    df['score_diff'] = df['score_diff'].str.rstrip(')')

    df = df.drop(columns = ['age'])

    
    df = adv_column_type_conversion(df)

    df['gameID'] = df['date'].apply(extract_date) + df["Tm"] + df["Opp"]

    df = df[col_order_adv]
    return df
    

In [301]:
df2 = clean_player_adv_games_log(df)

## Merge Advanced and Normal DFs

In [20]:
first = "Lebron"
last = "James"
season = 2005

df_adv = get_player_adv_season_games_log(first, last, season)
df_adv = clean_player_adv_games_log(df_adv)

df = get_player_season_games_log(first, last, season)
df = clean_player_games_log(df)

In [22]:
df_adv

,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,home,Opp,result,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,BPM
0,20041103CLEIND,1,1,2004-11-03 00:00:00,19,309,CLE,True,IND,L,...,6.3,33.5,1.1,4.6,19.8,30.3,103,105,20.9,7.4
1,20041104CLEMIA,2,2,2004-11-04 00:00:00,19,310,CLE,False,MIA,L,...,8.5,38.9,1.3,0.0,4.3,26.0,107,109,17.8,9.7
2,20041106CLEMIL,3,3,2004-11-06 00:00:00,19,312,CLE,False,MIL,L,...,10.4,32.9,3.6,0.0,11.9,34.6,104,103,23.8,9.2
3,20041109CLEATL,4,4,2004-11-09 00:00:00,19,315,CLE,False,ATL,W,...,13.9,17.4,4.7,3.2,11.9,25.8,111,83,22.5,8.7
4,20041110CLEPHO,5,5,2004-11-10 00:00:00,19,316,CLE,True,PHO,W,...,10.6,28.0,3.1,4.1,9.3,29.7,130,95,36.1,16.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,20050414CLENYK,78,76,2005-04-14 00:00:00,20,105,CLE,True,NYK,L,...,22.4,28.6,2.6,3.9,11.4,36.3,98,97,19.2,8.7
81,20050415CLEWAS,79,77,2005-04-15 00:00:00,20,106,CLE,False,WAS,L,...,7.5,23.1,1.1,0.0,2.7,32.1,121,131,25.7,5.5
82,20050417CLEDET,80,78,2005-04-17 00:00:00,20,108,CLE,False,DET,L,...,12.5,31.7,0.0,1.5,2.4,42.6,110,111,24.5,14.3
84,20050419CLEBOS,81,79,2005-04-19 00:00:00,20,110,CLE,True,BOS,W,...,10.5,22.9,5.8,0.0,9.3,31.0,121,92,25.7,8.4


In [24]:
df.columns

Index(['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days',
       'Tm', 'Opp', 'home', 'result', 'score_diff', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', 'PM'],
      dtype='object')

In [25]:
df_adv.columns

Index(['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days',
       'Tm', 'home', 'Opp', 'result', 'score_diff', 'GS', 'MP', 'TS%', 'eFG%',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg',
       'DRtg', 'GmSc', 'BPM'],
      dtype='object')

In [26]:
cols_post_merge = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 
                   'Tm', 'Opp', 'home', 'result', 'score_diff','GS', 'MP','FG', 'FGA',
                   'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
                   'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%', 'eFG%', 'ORB%', 'DRB%', 
                   'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg','DRtg', 'GmSc', 'BPM', 'PM']

In [61]:
def merge_gamelogs(df, df_adv):
    df_adv = df_adv.drop(columns=['tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days',
                          'Tm', 'home', 'Opp', 'result', 'score_diff', 'GS', 'MP','GmSc']) # Drop duplicated columns
    merged_df = pd.merge(df, df_adv, on='gameID', how='inner')
    return merged_df

In [54]:
test = merge_gamelogs(df, df_adv)

In [56]:
nan_rows = test.isna().any(axis=1)
print(nan_rows[nan_rows == True])


15    True
31    True
46    True
dtype: bool


In [42]:
test.columns

Index(['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days',
       'Tm', 'Opp', 'home', 'result', 'score_diff', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', 'PM', 'TS%', 'eFG%',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg',
       'DRtg', 'BPM'],
      dtype='object')

In [64]:
nba_players_list = pd.read_csv('src/raw_data/players_active_in_2000s.csv').set_index('Unnamed: 0')

In [92]:
# TODO extract into player specific helpers/util

def player_name_to_bbref_code(first, last):
    try:
        if len(last) >= 5:
            return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
        else:
            return "{}/{}{}01".format(last[:1], last, first[:2]).lower()
    except:
        print(f"Nice try {first} {last}")
        return None

In [75]:
player_name_to_bbref_code(first, last)

'a/abdulma01'

In [79]:
def player_name_to_bbref_code(first, last):
    if 
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()

In [81]:
first = "Nene"
last = None
test = player_name_to_bbref_code(first, last)
print(test)

None


In [82]:
nba_players = pd.read_csv("src/raw_data/nba_players_list.csv")

In [83]:
nba_players

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame
0,0,Alaa,Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke,False
1,1,Zaid,Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State,False
2,2,Kareem,Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA,True
3,3,Mahmoud,Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,False
4,4,Tariq,Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State",False
...,...,...,...,...,...,...,...,...,...,...,...
5103,15,Ante,Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",NaN,False
5104,16,Jim,Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,False
5105,17,Bill,Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne,False
5106,18,Ivica,Zubac,2017,2023,C,7-0,240.0,"March 18, 1997",NaN,False


In [85]:
row = nba_players.iloc[1982]
first = row['first_name']
last = row['last_name']
season = int(row['From'])


nan


In [93]:
player_name_to_bbref_code(first, last)

Nice try Nenê nan


In [95]:
df = pd.read_csv("src/raw_data/nba_player_codes.csv")

In [102]:
# TODO extract into player specific helpers/util

def player_name_to_bbref_code(first, last):
    try:
        if len(last) >= 5:
            return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
        else:
            return "{}/{}{}01".format(last[:1], last, first[:2]).lower()
    except:
        print(f"Nice try {first} {last}")
        return None

In [97]:
df

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame,bbref
0,0,Alaa,Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke,False,NaN
1,1,Zaid,Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State,False,NaN
2,2,Kareem,Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA,True,NaN
3,3,Mahmoud,Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,False,NaN
4,4,Tariq,Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State",False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5103,15,Ante,Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",NaN,False,NaN
5104,16,Jim,Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,False,NaN
5105,17,Bill,Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne,False,NaN
5106,18,Ivica,Zubac,2017,2023,C,7-0,240.0,"March 18, 1997",NaN,False,NaN


In [104]:
for index, row in df.iterrows():
    first = row['first_name']
    last = row['last_name']
    row['bbref'] = player_name_to_bbref_code(first, last)
    print(index, first, last, row['bbref'])

0 Alaa Abdelnaby a/abdelal01
1 Zaid Abdul-Aziz a/abdulza01
2 Kareem Abdul-Jabbar a/abdulka01
3 Mahmoud Abdul-Rauf a/abdulma01
4 Tariq Abdul-Wahad a/abdulta01
5 Shareef Abdur-Rahim a/abdursh01
6 Tom Abernethy a/abernto01
7 Forest Able a/ablefo01
8 John Abramovic a/abramjo01
9 Álex Abrines a/abrinál01
10 Precious Achiuwa a/achiupr01
11 Alex Acker a/ackeral01
12 Don Ackerman a/ackerdo01
13 Mark Acres a/acresma01
14 Bud Acton a/actonbu01
15 Quincy Acy a/acyqu01
16 Alvan Adams a/adamsal01
17 Don Adams a/adamsdo01
18 George Adams a/adamsge01
19 Hassan Adams a/adamsha01
20 Jaylen Adams a/adamsja01
21 Jordan Adams a/adamsjo01
22 Michael Adams a/adamsmi01
23 Steven Adams a/adamsst01
24 Rafael Addison a/addisra01
25 Bam Adebayo a/adebaba01
26 Deng Adel a/adelde01
27 Rick Adelman a/adelmri01
28 Jeff Adrien a/adrieje01
29 Arron Afflalo a/afflaar01
30 Ochai Agbaji a/agbajoc01
31 Maurice Ager a/agerma01
32 Mark Aguirre a/aguirma01
33 Blake Ahearn a/ahearbl01
34 Danny Ainge a/aingeda01
35 Matt Aitch 

In [106]:
df

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame,bbref
0,0,Alaa,Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke,False,NaN
1,1,Zaid,Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State,False,NaN
2,2,Kareem,Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA,True,NaN
3,3,Mahmoud,Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,False,NaN
4,4,Tariq,Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State",False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5103,15,Ante,Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",NaN,False,NaN
5104,16,Jim,Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,False,NaN
5105,17,Bill,Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne,False,NaN
5106,18,Ivica,Zubac,2017,2023,C,7-0,240.0,"March 18, 1997",NaN,False,NaN


In [123]:
import math

In [143]:
def name_to_code(row):
    first = row['first_name']
    last = row['last_name']
    if type(last) == float:
        return f"h/hilarne01"
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()


In [217]:
df = pd.read_csv("src/raw_data/nba_player_codes.csv")

In [120]:
if df.iloc[1982]['last_name'] == flo:
    print("HI")

NameError: name 'NaN' is not defined

In [136]:
for index, row in df.iterrows():
    # print(type(row['last_name']))
    if type(row['last_name']) == float:
        print(row)
        # print(row)  

Unnamed: 0                     238
first_name                    Nenê
last_name                      NaN
From                          2003
To                            2019
Pos                            F-C
Ht                            6-11
Wt                           250.0
birth_date      September 13, 1982
college                        NaN
hall_of_fame                   NaN
bbref                          NaN
Name: 1982, dtype: object


# Handling Duplicate bbref codes

This is a pain in the ass but necessary, some manual handling of the duplicate cases, goal is for nba_players.csv to have all 

In [248]:
duplicate_player_names = ['Bobby Wilson', 'Mark Davis', 'Jon Barry', 'Luke Jackson', 'Walker Russell', 'George Johnson',
                          'Mike Davis', 'Charles Jones', 'Marcus Williams', 'Bob Duffy', 'Sam Williams',
                          'Patrick Ewing', 'Charles Smith', 'Mike James', 'Michael Smith', 'Bobby Jones',
                          'Don Smith', 'Chris Johnson', 'Joe Barry Carroll', 'Eddie Johnson', 'Mark Jones',
                          'Gerald Henderson', 'Larry Johnson', 'Matt Guokas', 'Jim Paxson', 'Kenyon Martin',
                          'David Vaughn', 'Tony Mitchell', 'George King', 'Jack Turner', 'Dan Anderson',
                          'Cedric Henderson', 'Mike Dunleavy', 'Reggie Williams', 'Greg Smith', 'Johnny Davis',
                          'Dee Brown', 'Roger Brown', 'Freddie Lewis', 'Brandon Williams', 'Ken Johnson',
                          'Bill Bradley', 'Kenyon Martin Jr.', 'Sam Smith', 'Jeff Taylor', 'Tony Jackson',
                          'Chris Wright', 'Chris Smith']

In [391]:
df = pd.read_csv("src/raw_data/nba_player_codes.csv")
df.loc[:,'bbref'] = df.apply(name_to_code, axis=1)

In [392]:
# Goal of this is to properly populate all bbref codes, by end each row should have a unique bbref code

df = pd.read_csv('~/github/nba_data_scraper/src/raw_data/nba_players_list.csv') # Read in names of players
df.loc[:,'bbref'] = df.apply(name_to_code, axis=1) # First pass, leads to duplicates

value_counts = df.bbref.value_counts() # Get all value counts
duplicate_ids = value_counts[value_counts > 1].index.tolist() # get all ids that show multiple times

print(f"There are {len(duplicate_ids)} to correct, stage 1")

# Stage 1, fix duplicate ID's that have unique names
for idx, id in enumerate(duplicate_ids): # Iterate all over bbref codes that show in multiple rows
    temp = df[df['bbref'] == id].copy() # create a temp copy with just the rows for given id
    last = temp.head(1)['last_name'].item() # get common last name
    url = f"https://www.basketball-reference.com/players/{last[0].lower()}/"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    player_strings = soup.select('a[href^="/players/"]') 
    for index, row in temp.iterrows(): # iterate over players with identical bbref codes
        full = f"{row['first_name']} {row['last_name']}"
        out = soup.select('a[href^="/players/"]') 
        result = list(filter(lambda x: full in x, out))
        if len(result) == 1: # If unique name is using a duplicate ID, we can find and fix
            entry = result[0]
            full_name = str(entry.string)
            first, last = full_name.split(' ', maxsplit=1)
            code = entry['href'].lstrip("/players/").rstrip(".html")
            df.loc[(df['first_name'] == first) & (df['last_name'] == last), 'bbref'] = code
    print(f"{idx}/{len(duplicate_ids)}")

There are 236 to correct, stage 1
0/236
1/236
2/236
3/236
4/236
5/236
6/236
7/236
8/236
9/236
10/236
11/236
12/236
13/236
14/236
15/236
16/236
17/236
18/236
19/236
20/236
21/236
22/236
23/236
24/236
25/236
26/236
27/236
28/236
29/236
30/236
31/236
32/236
33/236
34/236
35/236
36/236
37/236
38/236
39/236
40/236
41/236
42/236
43/236
44/236
45/236
46/236
47/236
48/236
49/236
50/236
51/236
52/236
53/236
54/236
55/236
56/236
57/236
58/236
59/236
60/236
61/236
62/236
63/236
64/236
65/236
66/236
67/236
68/236
69/236
70/236
71/236
72/236
73/236
74/236
75/236
76/236
77/236
78/236
79/236
80/236
81/236
82/236
83/236
84/236
85/236
86/236
87/236
88/236
89/236
90/236
91/236
92/236
93/236
94/236
95/236
96/236
97/236
98/236
99/236
100/236
101/236
102/236
103/236
104/236
105/236
106/236
107/236
108/236
109/236
110/236
111/236
112/236
113/236
114/236
115/236
116/236
117/236
118/236
119/236
120/236
121/236
122/236
123/236
124/236
125/236
126/236
127/236
128/236
129/236
130/236
131/236
132/236
133/236
134/

In [439]:
# Stage 2, fix duplicate ID's that have non-unique names

dupe_names = pd.read_csv("~/github/nba_data_scraper/src/raw_data/duplicate_names.csv")

for index, row in dupe_names.iterrows():
    df.loc[(df['first_name'] == row['first_name']) & (df['last_name'] == row['last_name']) 
    & (df['From'] == int(row['From'])) & (df['To'] == int(row['To'])), 'bbref'] = row['bbref']
    
# Stage 3, Fuck you Tony Mitchell for needing more info, also for some reason Kenyon Martin Jr. == KJ Martin
df.loc[(df['first_name'] == "Tony") & (df['last_name'] == "Mitchell") & (df['college'] == 'Alabama'), 'bbref'] = "m/mitchto03"
df.loc[(df['first_name'] == "Tony") & (df['last_name'] == "Mitchell") & (df['college'] == 'University of North Texas'), 'bbref'] = "m/mitchto02"
df.loc[(df['first_name'] == "Kenyon") & (df['last_name'] == "Martin Jr."), 'first_name'] = "KJ"
df.loc[(df['first_name'] == "KJ") & (df['last_name'] == "Martin Jr."), 'last_name'] = "Martin"


df.to_csv("~/github/nba_data_scraper/src/raw_data/nba_player_codes.csv", index=False)

In [440]:
value_counts = df.bbref.value_counts() # Get all value counts
duplicate_ids = value_counts[value_counts > 1].index.tolist() # get all ids that show multiple times
print(value_counts)
print(duplicate_ids)

bbref
a/abdelal01    1
o/ojelese01    1
o/okorois01    1
o/okongon01    1
o/okogijo01    1
              ..
g/greense01    1
g/greenri01    1
g/greenmi01    1
g/greenlu01    1
z/zunicma01    1
Name: count, Length: 5108, dtype: int64
[]


In [441]:
df[(df['first_name'] == "Mahmoud") & (df['last_name'] == "Abdul-Rauf")]

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame,bbref
3,3,Mahmoud,Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,False,a/abdulma01


In [435]:
df[df.bbref == "b/barryjo01"]

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame,bbref
250,77,Jon,Barry,1993,2006,G,6-4,195.0,"July 25, 1969","University of the Pacific, Georgia Tech",False,b/barryjo01
720,56,Joe,Barry Carroll,1981,1991,C-F,7-0,225.0,"July 24, 1958",Purdue,False,b/barryjo01


# Finally Pulling player game logs!

Going to start here to deal with some of the edge cases:
- Need to handle PM only available starting 1996/97 (DONE)
- Need to handle differences going back further in df shape
- Determine how far back advanced stats go
- What happens if a player missed a season?

In [18]:
import bs4
import pandas as pd
from src.util.selenium_helper import get_webpage_html
from src.util.util_dicts import home_map, player_games_log_col_order

In [19]:
player_games_log_col_order = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 'Tm', 'Opp', 'home',
                              'result', 'score_diff', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA',
                              'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc',
                              'PM']  # Specific column order because cleaner

In [20]:
def get_player_season_games_log(season, code):
    url = f"https://www.basketball-reference.com/players/{code}/gamelog/{season}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    try:
        table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_basic')
        df = pd.read_html(str(table))[0]
    except ValueError:
        print(f"Unable to find {season} for {code}")
        df = None
    return df

In [21]:
# Specific for the player game log table
# TODO, investigate try/catch for this in case missing values?
def column_type_conversion(df):
    df.loc[:, 'tm_game_num'] = df['tm_game_num'].astype(int)
    df.loc[:, 'p_game_num'] = df['p_game_num'].astype(int)
    df.loc[:, 'home'] = df['home'].astype(bool)
    df.loc[:, 'GS'] = df['GS'].astype(bool)
    df.loc[:, 'FG'] = df['FG'].astype(int)
    df.loc[:, 'FGA'] = df['FGA'].astype(int)
    df.loc[:, 'FG%'] = df['FG%'].astype(float)
    df.loc[:, '3P'] = df['3P'].astype(int)
    df.loc[:, '3PA'] = df['3PA'].astype(int)
    df.loc[:, '3P%'] = df['3P%'].astype(float)
    df.loc[:, 'FT'] = df['FT'].astype(int)
    df.loc[:, 'FTA'] = df['FTA'].astype(int)
    df.loc[:, 'FT%'] = df['FT%'].astype(float)
    df.loc[:, 'ORB'] = df['ORB'].astype(int)
    df.loc[:, 'DRB'] = df['DRB'].astype(int)
    df.loc[:, 'TRB'] = df['TRB'].astype(int)
    df.loc[:, 'AST'] = df['AST'].astype(int)
    df.loc[:, 'STL'] = df['STL'].astype(int)
    df.loc[:, 'BLK'] = df['BLK'].astype(int)
    df.loc[:, 'TOV'] = df['TOV'].astype(int)
    df.loc[:, 'PF'] = df['PF'].astype(int)
    df.loc[:, 'PTS'] = df['PTS'].astype(int)
    df.loc[:, 'GmSc'] = df['GmSc'].astype(float)
    df.loc[:, 'PM'] = df['PM'].astype(float)  # E.G Lebron missing +/- leads to error when parsing
    df.loc[:, 'age_yrs'] = df['age_yrs'].astype(int)
    df.loc[:, 'age_days'] = df['age_days'].astype(int)
    df.loc[:, 'score_diff'] = df['score_diff'].astype(int)
    if type(df.iloc[0]['date']) == str:
        df.loc[:, 'date'] = pd.to_datetime(df['date'], format='mixed')
    return df

In [22]:
def extract_date(dt):  # To create gameID column
    return dt.date().strftime("%Y%m%d")

In [23]:
def clean_player_games_log(df, season):
    df = df[df['G'] != 'G'].copy()  # remove formatting rows
    column_rename = {'Rk': 'tm_game_num', 'G': 'p_game_num', 'Date': 'date', 'Age': 'age', 'Unnamed: 5': 'home',
                     'Unnamed: 7': 'result', '+/-': 'PM'}
    df.rename(columns=column_rename, inplace=True)  # Rename necessary columns

    df = df[df['GS'] != 'Inactive'].copy()  # Remove rows where the player was inactive for the game
    df = df[df['GS'] != 'Did Not Dress'].copy()  # Remove rows where player was not active (for player stats research would want to keep, but for ML not needed
    df = df[df['GS'] != 'Did Not Play'].copy()  # Remove rows where player was not active, mutual decision? lol
    df = df[df['GS'] != 'Not With Team'].copy()  # Remove rows where player was not with team?

    df.loc[:, 'home'] = df['home'].replace(home_map)  # Map home to a bool column, true = home

    # Split and clean up compound columns
    df[['age_yrs', 'age_days']] = df['age'].str.split('-', expand=True)
    df[['result', 'score_diff']] = df['result'].str.split('(', expand=True)
    df['score_diff'] = df['score_diff'].str.rstrip(')')
    df = df.drop(columns=['age'])

    if season < 1997:
        df['PM'] = np.nan

    df = column_type_conversion(df)  # Convert all columns to required type
    df['gameID'] = df['date'].apply(extract_date) + df["Tm"] + df["Opp"]  # Create gameID column

    df = df[player_games_log_col_order]  # Order because cleaner
    return df

In [24]:
# Specific for the player game log table
# TODO, investigate try/catch for this in case missing values?
def adv_column_type_conversion(df):
    df.loc[:, 'tm_game_num'] = df['tm_game_num'].astype(int)
    df.loc[:, 'p_game_num'] = df['p_game_num'].astype(int)
    df.loc[:, 'home'] = df['home'].astype(bool)
    df.loc[:, 'GS'] = df['GS'].astype(bool)
    df.loc[:, 'TS%'] = df['TS%'].astype(float)
    df.loc[:, 'eFG%'] = df['eFG%'].astype(float)
    df.loc[:, 'ORB%'] = df['ORB%'].astype(float)
    df.loc[:, 'DRB%'] = df['DRB%'].astype(float)
    df.loc[:, 'TRB%'] = df['TRB%'].astype(float)
    df.loc[:, 'AST%'] = df['AST%'].astype(float)
    df.loc[:, 'STL%'] = df['STL%'].astype(float)
    df.loc[:, 'BLK%'] = df['BLK%'].astype(float)
    df.loc[:, 'TOV%'] = df['TOV%'].astype(float)
    df.loc[:, 'USG%'] = df['USG%'].astype(float)
    df.loc[:, 'ORtg'] = df['ORtg'].astype(int)
    df.loc[:, 'DRtg'] = df['DRtg'].astype(int)
    df.loc[:, 'GmSc'] = df['GmSc'].astype(float)
    df.loc[:, 'BPM'] = df['BPM'].astype(float)
    df.loc[:, 'age_yrs'] = df['age_yrs'].astype(int)
    df.loc[:, 'age_days'] = df['age_days'].astype(int)
    df.loc[:, 'score_diff'] = df['score_diff'].astype(int)
    if type(df.iloc[0]['date']) == str:
        df.loc[:, 'date'] = pd.to_datetime(df['date'], format='mixed')
    return df

In [25]:
def get_player_adv_season_games_log(season, code):
    url = f"https://www.basketball-reference.com/players/{code}/gamelog-advanced/{season}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    try:
        table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_advanced')
        df = pd.read_html(str(table))[0]
    except ValueError:
        print(f"Unable to find {season} for {code}")
        df = None
    return df

In [26]:
def clean_player_adv_games_log(df):
    col_order = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 'Tm', 'home', 'Opp',
                 'result', 'score_diff', 'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%',
                 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM']

    df = df[df['G'] != 'G'].copy()  # remove formatting rows
    column_rename = {'Rk': 'tm_game_num', 'G': 'p_game_num', 'Date': 'date', 'Age': 'age', 'Unnamed: 5': 'home',
                     'Unnamed: 7': 'result'}
    df.rename(columns=column_rename, inplace=True)  # Rename necessary columns

    df = df[df['GS'] != 'Inactive'].copy()  # Remove rows where the player was inactive for the game
    df = df[df[
                'GS'] != 'Did Not Dress'].copy()  # Remove rows where player was not active (for player stats research would want to keep, but for ML not needed
    df = df[df['GS'] != 'Did Not Play'].copy()  # Remove rows where player was not active, mutual decision? lol
    df = df[df['GS'] != 'Not With Team'].copy()  # Remove rows where player was not with team?

    # NOTE: Will be more of these to drop, e.g. did not dress
    df.loc[:, 'home'] = df['home'].replace(home_map)  # Map home to a bool column, true = home

    # Split compound columns
    df[['age_yrs', 'age_days']] = df['age'].str.split('-', expand=True)
    df[['result', 'score_diff']] = df['result'].str.split('(', expand=True)
    df['score_diff'] = df['score_diff'].str.rstrip(')')

    df = df.drop(columns=['age'])

    df = adv_column_type_conversion(df)

    df['gameID'] = df['date'].apply(extract_date) + df["Tm"] + df["Opp"]

    df = df[col_order]
    return df

In [27]:
def merge_gamelogs(df, df_adv):
    df_adv = df_adv.drop(columns=['tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days',
                                  'Tm', 'home', 'Opp', 'result', 'score_diff', 'GS', 'MP',
                                  'GmSc'])  # Drop duplicate cols
    merged_df = pd.merge(df, df_adv, on='gameID', how='inner')
    return merged_df

In [28]:
def get_and_merge_all_logs(season, code):
    print(season, code)
    df = get_player_season_games_log(season, code)
    if df is None:
        return None
    df = clean_player_games_log(df, season)
    print("Normal pulled and clean")
    df_adv = get_player_adv_season_games_log(season, code)
    if df_adv is None:
        return None
    df_adv = clean_player_adv_games_log(df_adv)
    print("Adv pulled and clean")
    return merge_gamelogs(df, df_adv)

### Use edge case to handle PM difference

In [520]:
import os

In [517]:
player_codes = pd.read_csv("~/github/nba_data_scraper/src/raw_data/nba_player_codes.csv")

In [9]:
first = "Dale"
last = "Ellis"

In [10]:
player_codes[(player_codes.first_name == first) & (player_codes.last_name == last)]

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame,bbref
1290,58,Dale,Ellis,1984,2000,G-F,6-7,205.0,"August 6, 1960",Tennessee,False,e/ellisda01


In [ ]:
first = "Hassan"
last = "Adams"
start_year = 1984
end_year = 2000
code = "e/ellisda01"
df_master = pd.DataFrame()

if start_year >= 2000 or end_year >= 2000:
    print(f"{first} {last} played {start_year} to {end_year}")
    for season in range(start_year, end_year+1):
        print(season)
        df = get_and_merge_all_logs(season, code)
        if df is not None:
            df_master = pd.concat([df_master, df])


In [543]:
filepath = f"/Users/ericbackman/github/nba_data_scraper/src/raw_data/player_career_game_logs/{first}_{last}_game_logs.csv"

In [550]:
directory_path = '~/github/nba_data_scraper/src/raw_data/player_career_game_logs'

In [557]:
player_codes.bbref

0       a/abdelal01
1       a/abdulza01
2       a/abdulka01
3       a/abdulma02
4       a/abdulta01
           ...     
5103    ž/žižićan01
5104     z/zoetji01
5105     z/zopfbi01
5106    z/zubaciv01
5107    z/zunicma01
Name: bbref, Length: 5108, dtype: object

In [3]:
player_codes = pd.read_csv("~/github/nba_data_scraper/src/raw_data/nba_player_codes.csv")

In [5]:
from unidecode import unidecode

In [6]:
def remove_accents(text):
    return unidecode(text)

In [7]:
player_codes.bbref = player_codes.bbref.apply(remove_accents)

In [551]:
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]


In [7]:
player_names = pd.read_csv("src/raw_data/nba_player_codes.csv")

In [9]:
player_names[player_names['bbref'].str.len() < 11]

,Unnamed: 0,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame,bbref
7,7,Forest,Able,1957,1957,G,6-3,180.0,"July 27, 1932",Western Kentucky,False,a/ablefo01
15,15,Quincy,Acy,2013,2019,F-C,6-7,240.0,"October 6, 1990",Baylor,False,a/acyqu01
26,27,Deng,Adel,2019,2019,F,6-7,200.0,"February 1, 1997",Louisville,False,a/adelde01
31,32,Maurice,Ager,2007,2011,G,6-5,202.0,"February 9, 1984",Michigan State,False,a/agerma01
37,38,Henry,Akin,1967,1969,C-F,6-10,225.0,"July 31, 1944",Morehead State University,False,a/akinhe01
...,...,...,...,...,...,...,...,...,...,...,...,...
5074,7,Gabe,York,2022,2023,G,6-3,185.0,"August 2, 1993",Arizona,False,y/yorkga01
5086,19,Sun,Yue,2009,2009,G,6-9,205.0,"November 6, 1985",NaN,False,y/yuesu01
5096,8,Tony,Zeno,1980,1980,F,6-8,210.0,"October 1, 1957",Arizona State,False,z/zenoto01
5104,16,Jim,Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,False,z/zoetji01


In [12]:
filtered_df = player_names[~player_names['bbref'].str.contains("/")]